# Data Load and Exploration

### 📖 Instructions
The idea of this challenge is to identify in a better way your capacities to translate data into assets, we expect a good pipeline and solution that you can understand and translate. Select one of the below problems where it is affordable to create a REST API using python (scripts files) where we can identify the data pipeline/dataflow, but you can develop a MD file. Remember to Make it available over github 

#### Hint:
- Highlight variables or patterns using EDA
- Validate your functions
- Be clear with the pipeline
- Readme and Requirements files are expected




In [ ]:
import requests
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import xlrd
import warnings
warnings.filterwarnings('ignore')

from bs4 import BeautifulSoup
from urllib.parse import urljoin

#print(os.getcwd())
ROOT_DIR = os.path.dirname(os.path.dirname(os.path.abspath("ds_proj1")))
os.chdir(ROOT_DIR)
print(os.getcwd())

#from scripts import ptools

## Web scraping for data

In [ ]:
# detect links
web_url = 'https://www.cityobservatory.birmingham.gov.uk/@birmingham-city-council/purchase-card-transactions/'
response = requests.get(web_url)

# Create a BeautifulSoup object to parse the HTML
soup = BeautifulSoup(response.text, 'html.parser')
# Find all 'a' tags (links) in the HTML
links = soup.find_all('a')

# Extract links with the specified extension and pattern
all_links = [urljoin(web_url, link.get('href')) for link in links if link.get('href') and link.get('href').endswith('.xls')]
xls_links = list(filter(lambda link: '.datopian.' in link, all_links))

# Print the links ending with the specified extension
if xls_links:
    print(f"Links ending with {'.xls'} extension:")
    for idx, link in enumerate(xls_links, start=1):
        print(f"{idx}. {link}")

#print(xls_links)

In [ ]:
import re
from datetime import datetime

# List of URLs
urls = xls_links.copy()

# Función completa que integra la extracción de fechas de una lista de URLs y encuentra los últimos 6 meses disponibles
def extract_dates_and_find_last_6_months(urls):
    # Diccionario para mapear nombres de meses posiblemente incompletos a nombres completos
    months = {
        'jan': 'January', 'feb': 'February', 'mar': 'March', 'apr': 'April', 
        'may': 'May', 'jun': 'June', 'jul': 'July', 'aug': 'August', 
        'sep': 'September', 'oct': 'October', 'nov': 'November', 'dec': 'December'
    }

    # Función para extraer y formatear las fechas
    def format_year_month(url):
        file_name = url.split('/')[-1].lower()
        match = re.search(r'(\w+)(\d{2,4})', file_name.replace('-', ''))
        if match:
            month_part, year = match.group(1), match.group(2)
            year = f"20{year}" if len(year) == 2 else year

            for short_month, full_month in months.items():
                if short_month in month_part:
                    return f"{year} {full_month}"

            # Intento alternativo para extraer el mes
            month_match = re.search(r'(jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)', month_part)
            return f"{year} {months[month_match.group(0)]}" if month_match else None
        return None

    # Extraer y formatear las fechas de las URLs
    formatted_dates = [format_year_month(url) for url in urls]

    # Obtener la fecha actual
    current_date = datetime.now()
    current_year = current_date.year
    current_month = current_date.strftime("%B")

    # Función para obtener los índices de los últimos 6 meses disponibles
    def get_last_6_months_indices(dates, current_year, current_month):
        valid_dates = [(i, date) for i, date in enumerate(dates) if date is not None]
        date_objects = [(index, datetime.strptime(date, '%Y %B')) for index, date in valid_dates]
        date_objects.sort(key=lambda x: x[1], reverse=True)

        last_6_months_indices = []
        for index, date in date_objects:
            if date.year == current_year and date.strftime("%B") == current_month:
                last_6_months_indices.append(index)
                if len(last_6_months_indices) == 6:
                    break
            elif date < datetime(current_year, current_date.month, 1):
                last_6_months_indices.append(index)
                if len(last_6_months_indices) == 6:
                    break

        return last_6_months_indices

    # Encontrar y devolver los índices de los últimos 6 meses disponibles
    return get_last_6_months_indices(formatted_dates, current_year, current_month)


# Identificar los índices de los últimos 6 meses disponibles
last_6_months_indices = extract_dates_and_find_last_6_months(urls)


In [ ]:
last_6_months_urls = [urls[i] for i in last_6_months_indices]
last_6_months_urls



## Select time window for analysis
Last Six months available 

In [ ]:
# Time-period data 
import os
to_download = last_6_months_urls.copy()


def download_documents(links, destination_directory='downloads'):
    """
    Downloads documents from a list of direct download links.

    Parameters:
        links (list): List of direct download links.
        destination_directory (str): Directory where downloaded documents will be saved.

    Returns:
        list: List of paths to the downloaded documents.
    """
    # Create the destination directory if it doesn't exist
    if not os.path.exists(destination_directory):
        os.makedirs(destination_directory)

    document_paths = []

    for idx, link in enumerate(links, start=1):
        try:
            # Get the content of the document
            response = requests.get(link)
            response.raise_for_status()

            # Get the filename from the link
            file_name = os.path.join(destination_directory, f"document_{idx}.xls")

            # Save the content to a local file
            with open(file_name, 'wb') as file:
                file.write(response.content)

            document_paths.append(file_name)
            print(f"Document {idx} downloaded successfully: {file_name}")

        except requests.exceptions.RequestException as e:
            print(f"Error downloading document {idx} from {link}: {e}")

    return document_paths


downloaded_documents = download_documents(to_download, destination_directory= 'data/raw')


In [ ]:
import os
import pandas as pd
#print(os.getcwd())


def load_and_concatenate_xls(directory="data/raw/"):
    """
    Load and concatenate Excel (xls) files into a single DataFrame.

    Parameters:
        directory (str): Directory containing the xls files.

    Returns:
        pd.DataFrame: DataFrame containing combined information from all xls files.
    """
    frames = []

    # Iterate through all files in the directory
    for file in os.listdir(directory):
        if file.endswith(".xls"):
            # Build the full file path
            file_path = os.path.join(directory, file)

            # Read the Excel file into a DataFrame
            df = pd.read_excel(file_path)

            # Add the DataFrame to the list of DataFrames
            frames.append(df)

    # Concatenate all DataFrames into one
    result = pd.concat(frames, ignore_index=True)

    return result

# Call the function to load and concatenate the xls files
rawdata = load_and_concatenate_xls()

rawdata.to_csv('data/interim/raw_aggregate.csv', index=False, encoding='utf-8')
rawdata.sample(1000).to_csv('data/interim/to_test.csv', index=False, encoding='utf-8')

## Exploratory Data Analysis 

In [ ]:
# Creating raw data copy
df = rawdata.copy()

# Basic information about the dataset
data_info = df.info()

# Checking for missing values
missing_values = df.isnull().sum()

# Descriptive statistics for numerical features
numerical_stats = df.describe()
# Checking clean shape and nº of employees
rows, cols = df.shape 
director = df.Directorate.unique().shape[0]

# Checking unique values for categorical features
categorical_columns = df.select_dtypes(include=['object', 'category']).columns
unique_values = df[categorical_columns].nunique()

# Summarizing the analysis
print(data_info, missing_values, numerical_stats, unique_values.to_frame(name='Unique Values'))

# Visualize missing values with datetime on the y-axis
df['TRANS DATE'] = pd.to_datetime(df['TRANS DATE'])

plt.figure(figsize=(20, 6))
sns.heatmap(rawdata.set_index('TRANS DATE').isnull().transpose(), cmap='viridis',  cbar=False)
# format the x-axis tick labels as dates
plt.xticks(rotation=90)
#plt.gca().set_xticklabels(data['TRANS DATE'].data.strftime('%Y-%m'))
plt.title('Missing Values Heatmap')
plt.show()

In [ ]:
# Outlier detection for 'ORIGINAL GROSS AMT'
Q1 = df['ORIGINAL GROSS AMT'].quantile(0.25)
Q3 = df['ORIGINAL GROSS AMT'].quantile(0.75)
IQR = Q3 - Q1

# Defining bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filtering out the outliers
data_no_outliers = df[
    (df['ORIGINAL GROSS AMT'] >= lower_bound) & 
    (df['ORIGINAL GROSS AMT'] <= upper_bound)
]

# Plotting the original and outlier-removed distributions for comparison
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.boxplot(data=df['ORIGINAL GROSS AMT'])
plt.title('Original Data')

plt.subplot(1, 2, 2)
sns.boxplot(data=data_no_outliers['ORIGINAL GROSS AMT'])
plt.title('Data Without Outliers')

plt.tight_layout()
plt.show()

In [ ]:
# EDA: Creating visualizations for business-related insights

# Day of the Week
data_no_outliers['Day of Week'] = data_no_outliers['TRANS DATE'].dt.day_name()

# Month
data_no_outliers['Month'] = data_no_outliers['TRANS DATE'].dt.month_name()

# Part of Month (Early: 1-10, Mid: 11-20, Late: 21-end)
def part_of_month(day):
    if day <= 10:
        return 'Early'
    elif day <= 20:
        return 'Mid'
    else:
        return 'Late'

data_no_outliers['Part of Month'] = data_no_outliers['TRANS DATE'].dt.day.apply(part_of_month)

# Displaying the new features
data_no_outliers[['TRANS DATE', 'Day of Week', 'Month', 'Part of Month']].head()
plt.figure(figsize=(18, 12))

# Distribution of transactions over days of the week
plt.subplot(2, 2, 1)
sns.countplot(data=data_no_outliers, x='Day of Week', order=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])
plt.title('Transactions by Day of Week')
plt.xticks(rotation=45)

# Distribution of transactions over months
plt.subplot(2, 2, 2)
sns.countplot(data=data_no_outliers, x='Month', order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December'])
plt.title('Transactions by Month')
plt.xticks(rotation=45)

# Distribution of transactions in parts of the month
plt.subplot(2, 2, 3)
sns.countplot(data=data_no_outliers, x='Part of Month')
plt.title('Transactions by Part of Month')

# Distribution of 'ORIGINAL GROSS AMT'
plt.subplot(2, 2, 4)
sns.histplot(data_no_outliers['ORIGINAL GROSS AMT'], bins=30, kde=True)
plt.title('Distribution of Original Gross Amount')

plt.tight_layout()
plt.show()

In [ ]:
# Plotting the distribution of transactions for 'Directorate' and 'MERCHANT NAME'
plt.figure(figsize=(15, 10))

# Transactions Count by Directorate
plt.subplot(2, 1, 1)
data_no_outliers['Directorate'].value_counts().head(10).plot(kind='bar', color='skyblue')
plt.xticks(rotation=90)
plt.title('Top 10 Directorates by Number of Transactions')
plt.xlabel('Directorate')
plt.ylabel('Number of Transactions')

# Transactions Count by MERCHANT NAME
plt.subplot(2, 1, 2)
data_no_outliers['MERCHANT NAME'].value_counts().head(10).plot(kind='bar', color='skyblue')
plt.xticks(rotation=90)
plt.title('Top 10 Merchants by Number of Transactions')
plt.xlabel('Merchant Name')
plt.ylabel('Number of Transactions')

plt.tight_layout()
plt.show()

In [ ]:
# Calculating the average ORIGINAL GROSS AMT for each Directorate and Merchant
avg_gross_amt_by_directorate = data_no_outliers.groupby('DIRECTORATE')['ORIGINAL GROSS AMT'].mean().sort_values(ascending=False).head(10)
avg_gross_amt_by_merchant = data_no_outliers.groupby('MERCHANT NAME')['ORIGINAL GROSS AMT'].mean().sort_values(ascending=False).head(20)

# Plotting the average ORIGINAL GROSS AMT for Directorates and Merchants
plt.figure(figsize=(15, 10))

# Average Gross Amount by Directorate
plt.subplot(2, 1, 1)
aa = avg_gross_amt_by_directorate.plot(kind='barh', color='skyblue')
plt.xticks(rotation=90)
plt.title('Top 10 Directorates by Average Transaction Amount')
plt.xlabel('Directorate')
plt.ylabel('Average Transaction Amount (ORIGINAL GROSS AMT)')
aa.invert_yaxis()

# Average Gross Amount by Merchant
plt.subplot(2, 1, 2)
bb = avg_gross_amt_by_merchant.plot(kind='barh', color='skyblue')
plt.xticks(rotation=90)
plt.title('Top 20 Merchants by Average Transaction Amount')
plt.xlabel('Merchant Name')
plt.ylabel('Average Transaction Amount (ORIGINAL GROSS AMT)')
bb.invert_yaxis()

plt.tight_layout()
plt.show()

In [ ]:
top_directorate = data_no_outliers['DIRECTORATE'].value_counts().idxmax()

# Filtering data for the top Directorate category
top_directorate_data = data_no_outliers[data_no_outliers['DIRECTORATE'] == top_directorate]

# Finding the top 10 merchants in this Directorate category
top_merchants_in_top_directorate = top_directorate_data['MERCHANT NAME'].value_counts().head(10)

# Plotting the top 10 merchants in the top Directorate category
plt.figure(figsize=(10, 6))
ax = top_merchants_in_top_directorate.plot(kind='barh', color='skyblue')
plt.title(f'Top 10 Merchants in {top_directorate} Directorate')
plt.xlabel('Number of Transactions')
plt.ylabel('Merchant Name')
ax.invert_yaxis()
plt.show()